<a href="https://colab.research.google.com/github/thomasbnr/Detection_depression/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des librairies

In [ ]:
import pandas as pnd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import files
from nltk.stem import WordNetLemmatizer

In [ ]:
tweet = pnd.read_csv('sentiment_depressive.csv',delimiter=',',encoding="utf-8",header=[0],index_col=[0])
tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   59997 non-null  object
 1   label   60000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [ ]:
nbr_label = tweet["label"].value_counts()
print(nbr_label)

1    30000
0    30000
Name: label, dtype: int64


Import de la bibliothèque contenant les mots vides (ou stopwords)

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopwords = stopwords.words('english')
print(stopwords)
print(len(stopwords))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Pré traitement

Suppression des URLs

In [ ]:
# Supprimer les URLs commençant par "https://", "http://", ou "pic.twitter.com/"
without_urls = "(https?://\S+|pic\.twitter\.com/\S+)"

tweet = tweet.applymap(lambda x: re.sub(without_urls, ' ', str(x)))
tweet.sample(5)

,tweet,label
6162,depression gym mix well me. unmotivated,1
27072,"lingerie won’t cure chronic depression, i’m go...",1
3344,Calm equanimity depression.,1
25864,"It triggering me, friends. I anxiety depression,",1
7226,I hate place. I hate job. I hate current life....,1


Suppression des mentions d'utlisateurs (@)

In [ ]:
remove_mentions = "@\S+"
tweet = tweet.applymap(lambda x: re.sub(remove_mentions , ' ', str(x)))
tweet.sample(5)

,tweet,label
56592,"No, it was from April 24 and came across it ...",0
30766,so whats ur buddy doing?,0
55886,"i would say yes, as the po counts saturday a...",0
8190,"Ppl often mistaken lonely quiet lot. I talk, y...",1
41679,went well. it's been a tiring and exciting w...,0


Suppression des caractères non alphanumériques

In [ ]:
non_alphanumeric_character = "[^A-Za-z0-9]+"
tweet = tweet.applymap(lambda x: re.sub(non_alphanumeric_character, ' ', str(x)))

Suppression des lignes vides

In [ ]:
empty_lines = tweet.loc[tweet['tweet'].str.strip().eq('')]
tweet = tweet.drop(empty_lines.index)
tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59207 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   59207 non-null  object
 1   label   59207 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


Suppresion des mots de 2 carctères ou moins

In [ ]:
tweet['tweet'] = tweet['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59207 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   59207 non-null  object
 1   label   59207 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


Tokénisation

In [ ]:
nltk.download('punkt')

def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    return tokens

tweet['tokens'] = tweet['tweet'].apply(tokenize_text)

#Supppression des stopwords
tweet['tokens'] = tweet['tokens'].apply(lambda x: ' '.join([item for item in x if item not in stopwords]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tweet["tokens"].head(10)

0              one packet artificial sweetener creamer
1                                      depression away
2             afraid loneliness yes lonely worse alone
3    bad day depressed still take alls animal pictu...
4     nowplaying joey aces deal ready lonely oak radio
5    afraid loneliness nope sometimes lonely better...
6    well accidentally wake 30pm today slept hours ...
7    scarlet decided part shane gang stated lonely ...
8                       accidentally yesterday hfhdhdj
9        give dog insulin shot tomorrow lowkey nervous
Name: tokens, dtype: object

Lemmatisation

In [ ]:
nltk.download('wordnet')

def lemmatize_text(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens.split()]
    return lemmatized_tokens

tweet["tokens"] = tweet["tokens"].apply(lambda x: ' '.join(lemmatize_text(x)))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
tweet["tokens"].head(10)

0              one packet artificial sweetener creamer
1                                      depression away
2             afraid loneliness yes lonely worse alone
3    bad day depressed still take alls animal pictu...
4      nowplaying joey ace deal ready lonely oak radio
5    afraid loneliness nope sometimes lonely better...
6    well accidentally wake 30pm today slept hour c...
7    scarlet decided part shane gang stated lonely ...
8                       accidentally yesterday hfhdhdj
9        give dog insulin shot tomorrow lowkey nervous
Name: tokens, dtype: object

In [ ]:
tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59207 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   59207 non-null  object
 1   label   59207 non-null  object
 2   tokens  59207 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


In [ ]:
export_csv = tweet.to_csv(r'pre_processed_data.csv')

files.download('pre_processed_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>